In [1]:
import json
from openai import OpenAI
from dotenv import load_dotenv
from tqdm import tqdm
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [2]:
def get_requirements_from_text(full_text: str):
    prompt = f"""
당신은 회의록에서 **시스템 개발자 관점에서 직접적으로 필요한 요구사항**을 추출하고 구조화하는 전문가입니다.
추출된 요구사항은 시스템의 설계, 개발, 테스트, 배포, 운영에 직접적으로 활용될 수 있어야 합니다.

각 요구사항은 다음 속성을 포함하는 JSON 객체로 정의되어야 합니다.
-   **id**: 고유 식별자 (예: "FUNC-001", "NFR-001"). (RFP 원문에 ID가 있다면 그것을 우선 사용)
-   **type**: 요구사항의 유형. 다음 중 선택: "기능적", "비기능적"(성능, 보안, 사용성, 안정성, 호환성 등 포함), "데이터", "인터페이스", "기술적 제약", "운영환경", "테스트 조건".
-   **description**: 개발자가 이해할 수 있도록 요구사항에 대한 명확하고 간결한 설명. 원본 텍스트의 핵심을 반영하되, 시스템의 행동, 속성, 제약 등을 명시.
-   **acceptance_criteria**: 이 요구사항이 충족되었음을 개발팀 또는 QA팀이 검증할 수 있는 구체적이고 측정 가능한 기준. 1~3개의 명확한 문장으로 서술. (예: "[특정 입력] 시 시스템은 [예상 출력/행동]을 보여야 한다", "응답 시간은 평균 X초 이내여야 한다", "[특정 표준]을 준수해야 하며, [검증 방법]으로 확인"). 추론이 매우 어려우면 "구체적인 검증 방법은 세부 설계 시 정의" 등으로 명시.
-   **priority**: 요구사항의 중요도. "필수", "높음", "중간", "낮음" 중 선택. 텍스트에 명시 없으면 "필수"로 간주.
-   **responsible_module**: 이 요구사항이 주로 구현되거나 영향을 미칠 시스템의 모듈 또는 구성요소. (예: "인증 모듈", "데이터베이스 스키마", "API 게이트웨이"). 추론 어려우면 "전체 시스템" 또는 "미정".
-   **source_pages**: 이 요구사항이 발견된 원본 PDF 페이지 번호 리스트.
-   **raw_text_snippet**: 추출 근거가 된 핵심 원본 텍스트 조각.

응답은 JSON 배열 형태 (`{{"requirements_extracted": [...]}}`)로 제공해야 합니다. 요구사항이 없다면 빈 배열 `[]`을 값으로 포함하여 반환하십시오.
**목차, 제안서 작성 요령, 업체 선정 기준, 일반적인 계약 조건 등 시스템의 기술적 구현과 직접 관련 없는 내용은 요구사항으로 추출하지 마십시오.**

다음은 회의록 텍스트입니다. 여기서 직접적인 요구사항을 추출하십시오:

\"\"\"
{full_text}
\"\"\"
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "당신은 시스템 분석 전문가이며, 요구사항을 추출하는 역할입니다."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return json.dumps({"error": str(e)})


In [ ]:
from docx import Document
import pandas as pd
from pypdf import PdfReader

In [9]:
input_txt = "./data/대한체육회_회의록.txt"
input_pdf = "./data/대한체육회_학습운영시스템_추가_요구사항_제안_회의_텍스트_변환본.pdf"
input_docs = "./data/대한체육회 학습운영시스템 추가 요구사항 제안 회의 텍스트 변환본.docx"
input_xlsx = "./data/대한체육회 학습운영시스템 추가 요구사항 제안 회의 텍스트 변환본.xlsx"

In [ ]:
def read_docx(file_path: str) -> str:
    doc = Document(file_path)
    full_text = [para.text for para in doc.paragraphs]
    return "\n".join(full_text)

def read_pdf(file_path: str) -> str:
    reader = PdfReader(file_path)
    texts = []
    for page in reader.pages:
        texts.append(page.extract_text() or "")
    return "\n".join(texts)

def read_excel(file_path: str) -> str:
    ext = os.path.splitext(file_path)[1].lower()
    if ext == ".xls":
        df = pd.read_excel(file_path, engine='xlrd')  # .xls 전용
    else:
        df = pd.read_excel(file_path, engine='openpyxl')  # .xlsx 전용
    text = df.astype(str).agg(' '.join, axis=1).str.cat(sep='\n')
    return text

def read_text_or_pdf_or_docx_or_excel(file_path: str) -> str:
    ext = os.path.splitext(file_path)[1].lower()
    if ext == ".txt":
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()
    elif ext == ".pdf":
        return read_pdf(file_path)
    elif ext == ".docx":
        return read_docx(file_path)
    elif ext in [".xls", ".xlsx"]:
        return read_excel(file_path)
    else:
        raise ValueError("지원하지 않는 파일 형식입니다. txt, pdf, docx, xls, xlsx만 지원합니다.")

In [13]:
try:
    full_text = read_text_or_pdf_or_docx_or_excel(input_xlsx)
except Exception as e:
    print(f"파일 읽기 오류 발생: {e}")
    full_text = None

if full_text:
    result = get_requirements_from_text(full_text)
    print(result)
    with open("extracted_additional_requirements.json", "w", encoding="utf-8") as f:
        f.write(result)

회의록에서 추출한 요구사항은 다음과 같습니다:

```json
{
  "requirements_extracted": [
    {
      "id": "FUNC-001",
      "type": "기능적",
      "description": "AI 기반 맞춤 학습 추천 시스템을 구현하여 수강자의 학습 이력을 분석해 개인별 강의를 추천한다.",
      "acceptance_criteria": "수강 이력 기반으로 추천 강의가 정확히 제공되고, 추천 결과가 사용자의 피드백에 따라 개선되는지 검증한다.",
      "priority": "필수",
      "responsible_module": "추천 시스템 모듈",
      "source_pages": [1],
      "raw_text_snippet": "FUNC-001 기능적 AI 기반 맞춤 학습 추천 시스템 구현: 수강자의 학습 이력을 분석해 개인별 강의를 추천한다. 수강 이력 기반으로 추천 강의가 정확히 제공되고, 추천 결과가 사용자의 피드백에 따라 개선되는지 검증."
    },
    {
      "id": "FUNC-002",
      "type": "기능적",
      "description": "챗봇 기능을 도입하여 학습자가 시스템 사용 시 기본 문의를 챗봇으로 처리할 수 있도록 한다.",
      "acceptance_criteria": "학습자가 챗봇을 통해 80% 이상의 기본 문의에 대해 즉시 응답을 받는지 확인한다.",
      "priority": "필수",
      "responsible_module": "챗봇 모듈",
      "source_pages": [1],
      "raw_text_snippet": "FUNC-002 기능적 챗봇 기능 도입: 학습자가 시스템 사용 시 기본 문의를 챗봇으로 처리할 수 있도록 한다. 학습자가 챗봇을 통해 80% 이상의 기본 문의에 대해 즉시 응답을 받는지 확인."
    },
    {
      "id": "FUNC